# Advanced Cirq Functionality

In this bonus part, we will explore 3 more advanced use cases of Cirq:
 * [Protocols](https://quantumai.google/cirq/protocols)
 * [Custom Gates](https://quantumai.google/cirq/custom_gates)
 * [Operators & Observables](https://quantumai.google/cirq/operators)

These can offer a more in-depth look at what Cirq is capable of.
You will not have to implement anything in this part.

Follow the code and observe the behaviour.

## Exercises

In [ ]:
try:
    import cirq
except ImportError:
    %pip install --quiet cirq
    import cirq
import numpy as np

### 1. Protocols

Cirq's protocols are very similar concept to Python's built-in protocols.
For example, behind all the for loops and list comprehensions you can find the *Iterator* protocol.
As long as an object has the `__iter__()` method that returns an iterator object, it has iterator support.
An iterator object has to define `__iter__()` and `__next__()` methods, that defines the iterator protocol.

A canonical Cirq protocol example is the `unitary` protocol that allows to check the unitary matrix of values that support the protocol by calling `cirq.unitary(val)`

In [ ]:
print(cirq.X)
print("cirq.X unitary:\n", cirq.unitary(cirq.X))

a, b = cirq.LineQubit.range(2)
circuit = cirq.Circuit(cirq.X(a), cirq.Y(b))
print(circuit)
print("circuit unitary:\n", cirq.unitary(circuit))

#### What is a protocol?

A protocol is a combination of the following two items:

 * A `SupportsXYZ` class, which defines and documents all the functions that need to be implemented in order to support that given protocol
 * The entrypoint function(s), which are exposed to the main cirq namespace as `cirq.xyz()`

The following family of protocols is an important and frequently used set of features of Cirq.
They are, in the order of increasing generality:
 * `unitary`
 * `kraus`
 * `mixture`


The `unitary` protocol is the least generic, as only unitary operators should implement it.
The `cirq.unitary` function returns the matrix representation of the operator in the computational basis.
We saw an example of the unitary protocol above, but let's see the unitary matrix of the Pauli-Y operator as well: 

In [ ]:
print(cirq.unitary(cirq.Y))

The `kraus` representation is the operator sum representation of a quantum operator (a channel).
These matrices are required to satisfy the trace preserving condition.

The `cirq.kraus` returns a tuple of numpy arrays, one for each of the Kraus operators.
In the simplest case of a unitary operator, `cirq.kraus` returns a one-element tuple with the same unitary as returned by `cirq.unitary`:

In [ ]:
print(cirq.kraus(cirq.Y))
print(cirq.unitary(cirq.Y))
print(cirq.has_kraus(cirq.Y))

The `mixture` protocol should be implemented by operators that are unitary-mixtures.
These probabilistic operators are represented by a list of tuples (, ), where each unitary effect occurs with a certain probability.
Probabilities are a Python float between 0.0 and 1.0, and the unitary matrices are `numpy` arrays.

In [ ]:
probabilistic_x = cirq.X.with_probability(.3)

for p, op in cirq.mixture(probabilistic_x):
    print(f"probability: {p}")
    print("operator:")
    print(op)

### 2. Custom Gates

Standard gates such as Pauli gates and CNOTs are defined in `cirq.ops`.
To use a unitary which is not a standard gate in a circuit, custom gates can be created.

#### General Pattern

Gates are classes in Cirq.
To define custom gates, we inherit from a base gate class and define a few methods.

The general workflow is to:
 1. Inherit from `cirq.Gate`
 2. Define one of the `_num_qubits_` or `_qid_shape_` methods
 3. Define one of the `_unitary_` or `_decompose_` methods


Here, we define a gate which corresponds to the unitary:

<img src="./res/custom_gate.png" alt="Custom Gate" width="200"/>

In [ ]:
class MyGate(cirq.Gate):
    def __init__(self):
        super(MyGate, self)

    def _num_qubits_(self):
        return 1

    def _unitary_(self):
        return np.array([
            [1.0,  1.0],
            [-1.0, 1.0]
        ]) / np.sqrt(2)

    def _circuit_diagram_info_(self, args):
        return "G"

Create the circuit and simulate it

In [ ]:
"""Use the custom gate in a circuit."""
circ = cirq.Circuit(
    MyGate().on(cirq.LineQubit(0))
)

print("Circuit with custom gates:")
print(circ)

sim = cirq.Simulator()

res = sim.simulate(circ)
print(res)

We can also have have multiple qubit custom gates which also take parameters.
In order to do this, we need to modify the number of qubits.
Moreover, we need to add the parameters to the `__init__` constructor.
We will create the Rotation gate with two parameters:

In [ ]:
"""Define a custom gate with two parameters."""
class RotationGate(cirq.Gate):
    def __init__(self, theta, pfi):
        super(RotationGate, self)
        self.theta = theta
        self.pfi = pfi

    def _num_qubits_(self):
        return 2

    def _unitary_(self):
        return np.array([
            [np.cos(self.theta), np.sin(self.pfi)],
            [np.sin(self.pfi), -np.cos(self.theta)]
        ]) / np.sqrt(2)

    def _circuit_diagram_info_(self, args):
        return f"Rt({self.theta})", f"Rp{self.pfi})"

In [ ]:
"""Use the custom gate in a circuit."""
circ = cirq.Circuit(
    RotationGate(theta=0.1, pfi=0.2).on(*cirq.LineQubit.range(2))
)

print("Circuit with a custom rotation gate:")
print(circ)

### 3. Operators & Observables

Quantum operations (or just operators) include unitary gates, measurements, and noisy channels.
Operators that act on a given set of qubits implement `cirq.Operation` which supports the Kraus operator representation.

As we have already discussed the other operators, we will only focus on *noisy channels*.

One example of *noisy channels* is the `DepolarizingChannel`.
Observe the channel and the kraus operations.
Also observe we can add noise to a Circuit.

In [ ]:
depo_channel = cirq.DepolarizingChannel(p=0.01, n_qubits=1)
print(depo_channel)

kraus_ops = cirq.kraus(depo_channel)
print(f"Kraus operators of {depo_channel} are:", *[op.round(2) for op in kraus_ops], sep="\n\n")

In [ ]:
q1 = cirq.LineQubit(0)

circuit = cirq.Circuit(
    cirq.H(q1),
    cirq.depolarize(p=0.01).on(q1),
    cirq.measure(q1)
)

print(circuit)